# Summer Data Scientist Data Assessment
## Crime and Education Lab New York
### Jesica Maria Ramirez Toscano

### *Part 1: Variable Creation*

In [5]:
import pandas as pd
import numpy as np
arrests = pd.read_csv('arrests.csv')
demo = pd.read_csv('demo.csv')
demo['bdate'] = pd.to_datetime(demo['bdate'], utc=False)
arrests['arrest_date'] = pd.to_datetime(arrests['arrest_date'], utc=False)

1. We filter the arrest to the ones that occurred post-implementation.  
2. Since we need information about past arrests and potential felony re-arrests, we merge the post-arrests with the total arrests by person_id. So each arrest will be linked to a post-arrest of the same individual.
> Note: **arrest_post** refers to the data of arrests post-implementation.  **tr** refers to the merged data of arrests_post with the all of the arrests. So each arrest in this data set is linked to a post-arrest of the same individual. 

In [11]:
arrests_post = arrests[arrests['arrest_date'] >= '2010-01-01'].copy()
tr = pd.merge(arrests,
              arrests_post.rename(columns={'arrest_date':'date_post', 
                                            'arrest_id':'aid_post', 
                                            'law_code':'code_post'}), 
              on='person_id')

3. We create different tables to obtain the number of prior misdemeanor arrests and felony arrests in the last 2 years and 6 months.

In [12]:
twoyear = tr[(tr['arrest_date'] >= tr['date_post']-pd.DateOffset(years=2)) & (tr['arrest_id'] != tr['aid_post'])]
sixmonth = tr[(tr['arrest_date'] >= tr['date_post'] - pd.DateOffset(months=6)) & (tr['arrest_id'] != tr['aid_post'])]
twoyear =  twoyear.groupby(['aid_post', 'law_code']).size().unstack().reset_index().fillna(0)
twoyear.rename(columns = {'aid_post':'arrest_id', 'felony': 'fel_2y', 'misdemeanor': 'mis_2y'}, inplace=True)
sixmonth = sixmonth.groupby(['aid_post', 'law_code']).size().unstack().reset_index().fillna(0)
sixmonth.rename(columns = {'aid_post':'arrest_id', 'felony': 'fel_6m', 'misdemeanor': 'mis_6m'}, inplace=True)

>So for the table **twoyear**, we have the post_arrests variable with the number of prior arrests of felony and misdemeanor in the last two years.

In [9]:
twoyear

law_code,arrest_id,fel_2y,mis_2y
0,000192be,0.0,1.0
1,000316a6,1.0,3.0
2,0005e711,1.0,3.0
3,000fca1a,1.0,3.0
4,00102ecf,0.0,3.0
...,...,...,...
19068,fff23454,1.0,4.0
19069,fff2c58f,0.0,3.0
19070,fff4f37a,1.0,0.0
19071,fff50975,1.0,0.0


4. To create the felony re-arrest binary variable, we need information about the potential future felony arrest of that invidual. So first, we create a table called **year_ahead** using the **tr** dataset.

In [20]:
year_ahead = tr[(tr['arrest_date'] >= tr['date_post']) & (tr['arrest_id'] != tr['aid_post'])]
year_ahead = year_ahead[year_ahead['arrest_date'] <= year_ahead['date_post'] + pd.DateOffset(years=1)]
year_ahead = year_ahead.groupby(['aid_post', 'law_code']).size().unstack().reset_index().fillna(0)
year_ahead.rename(columns = {'aid_post':'arrest_id', 'felony': 'felony-' }, inplace=True)

In [21]:
year_ahead[['arrest_id', 'felony']]

law_code,arrest_id,felony
0,000192be,0.0
1,000fca1a,0.0
2,00102ecf,0.0
3,001a4d40,0.0
4,002183f7,0.0
...,...,...
8975,ffd580fe,0.0
8976,ffe20b13,1.0
8977,fff23454,0.0
8978,fff2c58f,0.0


>With this table, we can create a binary variable of re_arrest (1 if the individual has one or more felony arrests during a one-year period following the arrest, 0 if the individual has no felony re-arrest)

In [16]:
year_ahead['re_arrest'] = np.where(year_ahead['felony'] > 0,1,0)